## Ne fonctionne pas avec: 
- 'MTGNN'
- 'DCGRU'
- 'LSTM'
- 'RNN'
- 'CNN'
- 'GRU'

In [27]:
import os
import sys
import torch
import numpy as np
import random
import pandas as pd  

# === FIXE LA SEED POUR REPRODUCTIBILITÉ ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# === CHEMIN ET IMPORTS ===
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config,train_the_config,get_ds
from examples.benchmark import local_get_args
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from trainer import Trainer

# === PARAMÈTRES GÉNÉRAUX ===
SEED = 42
EPOCHS = 1  # une seule epoch
station = ['BEL','PAR','AMP','SAN','FLA']
training_mode_to_visualise = ['test']

model_names = ['STGCN','ASTGCN']   # 'DCGRU','MTGNN','LSTM','RNN','CNN','GRU'
dataset_for_coverage = ['subway_in', 'netmob_image_per_station']
dataset_names= ["subway_in","subway_out"] #     ["subway_in", "netmob_POIs"] #    ["subway_in"]

def get_modification(dataset_names):
    # Définir la base de la modification
    modification = {
        'epochs': EPOCHS,
        'lr': 5e-5,
        'weight_decay': 0.05,
        'dropout': 0.15,
        'scheduler': None,
        'adj_type': 'corr',
        'threshold': 0.7,
        'stacked_contextual': True,
        'target_data': 'subway_in',
        'compute_node_attr_with_attn': False,
    }

    if "netmob_POIs" in dataset_names:
        modification.update({
            'NetMob_only_epsilon': True,
            'NetMob_selected_apps': ['Google_Maps'],
            'NetMob_transfer_mode': ['DL'],
            'NetMob_selected_tags': ['station_epsilon100'],
            'NetMob_expanded': ''
        })
    return modification

def load_inputs(model_name,dataset_names):
    # Init args
    modification = get_modification(dataset_names=dataset_names)
    args_init = local_get_args(model_name,args_init=None,dataset_names=dataset_names,dataset_for_coverage=dataset_for_coverage,modification=modification)

    # Load ds
    ds,args,trial_id,save_folder,df_loss = get_ds(modification=modification,args_init=args_init)
    return ds,args,trial_id,save_folder

# === FONCTION POUR UNE CONFIG SPÉCIFIQUE ===
def run_test(model_names, dataset_names):
    set_seed(SEED)
    df = pd.DataFrame()
    ds,args,trial_id,save_folder = load_inputs(model_names[0],dataset_names)
    for model_name in model_names:
        print(f"\n=== TESTING {model_name} on {dataset_names} ===")
        args.model_name = model_name
        model = load_model(ds, args)
        optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
        trainer = Trainer(ds,model,args,optimizer,loss_function,scheduler = scheduler,show_figure = False,trial_id = trial_id, fold=0,save_folder = save_folder)
        trainer.train_and_valid(normalizer = ds.normalizer, mod = 1000,mod_plot = None) 

        mse_test = trainer.performance['test_metrics']['mse']
        mse_valid = trainer.performance['valid_metrics']['mse']
        df = pd.concat([df,pd.DataFrame({'mse_test': [mse_test], 'mse_valid': [mse_valid]}, index=[model_name])], axis=0)

    print("=== TEST COMPLETED ===")
    return ds,trainer,df


# === LANCEMENT GLOBAL ===
if __name__ == "__main__":
    ds,last_trainer,df = run_test(model_names, dataset_names)
    
    # Afficher le DataFrame des résultats
    print("\n=== RÉSULTATS ===")
    display(df)
    
    # Créer un DataFrame de référence pour vérification
    checking = pd.DataFrame({
        'mse_test': [5988.421875, 48209.429688],
        'mse_valid': [6884.651855, 71904.921875]
    }, index=['STGCN', 'ASTGCN'])

    print("\n=== HAS TO BE EQUAL TO: ===")
    display(checking)

    print("\n=== ABSOLUTE DIFFERENCE BETWEEN BOTH DF: ===")
    display(abs(checking-df))

>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
T_subway_out:  torch.Size([7392, 40])
vision_input_type POIs
vision_model_name None
target_ds.U_valid torch.Size([940, 40, 7])
contextual_ds.U_valid torch.Size([940, 40, 7])
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940
Invalid dates within this fold: 481
Considered Spatial-Unit:  Index(['AMP', 'BEL', 'BRO', 'COR', 'CUI', 'CUS', 'FLA', 'GOR', 'BLA', 'GRA',
       'GUI', 'GIL', 'HEN', 'HOT', 'LAE', 'MAS', 'MER', 'LUM', 'PRY', 'PER',
       'SAN', 'SAX', 'VMY', 'JEA', 'BON', 'CHA', 'VAI', 'VEN', 'MAC', 'GAR',
       'FOC', 'REP', 'GER', 'DEB', 'JAU', 'CPA', 'CRO', 'PAR', 'SOI', 'OGA'],
      dtype='object', name='COD_TRG')

Init Dataset: 'torch.Size([4597, 40]) with 183880 Total nb of elements and 0 Nan values


/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


epoch: 0 
 min\epoch : 0.03

Training Throughput:1082.61 sequences per seconds
>>> Training complete in: 0:00:02.109553
>>> Training performance time: min 0.02060079574584961 avg 0.028305530548095703 seconds (+/- 0.0023735269131620945)
>>> Loading performance time: min 0.00013017654418945312 avg 0.00024142409815932766 seconds (+/- 4.2310904558851955e-05)
>>> Forward performance time: 0.015200282588149563 seconds (+/- 0.0017213593427379426)
>>> Backward performance time: 0.01320254441463586 seconds (+/- 0.0016452277229920805)
>>> Plotting performance time: nan seconds (+/- nan)
>>> Saving performance time: 0.012246370315551758 seconds (+/- 0.0)
>>> PI-tracking performance time: nan seconds (+/- nan)
>>> Scheduler-update performance time: nan seconds (+/- nan)
>>> Validation time: 0:00:00.354209
Proportion of time consumed for Loading: 1.4%
Proportion of time consumed for Forward: 29.9%
Proportion of time consumed for Backward: 25.8%
Proportion of time consumed for Plotting: 0.0%
Proport

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



=== RÉSULTATS ===


,mse_test,mse_valid
STGCN,5988.421875,6884.651855
ASTGCN,48209.429688,71904.921875



=== HAS TO BE EQUAL TO: ===


,mse_test,mse_valid
STGCN,5988.421875,6884.651855
ASTGCN,48209.429688,71904.921875



=== ABSOLUTE DIFFERENCE BETWEEN BOTH DF: ===


,mse_test,mse_valid
STGCN,0.000000e+00,4.687499e-07
ASTGCN,4.999965e-07,0.000000e+00
